In [1]:
from collections import OrderedDict
from conllu import parse 
from enum import Enum

In [2]:
PATH = "UD_Ukrainian-IU"

with open(PATH + "/uk_iu-ud-train.conllu", "r") as f:
    train_trees = parse(f.read())

with open(PATH + "/uk_iu-ud-dev.conllu", "r") as f:
    test_trees = parse(f.read())

In [3]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"

In [4]:
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [5]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

In [6]:
def extract_features(stack, queue):
    features = dict()

    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
         # фічі для слова в стеці
        if stack_top["feats"] != None:
            features["s0-feats"] = "_".join([f for f in stack_top["feats"].values()])
        else:
            features["s0-feats"] = False
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
        if stack[-2]["feats"] != None:
            features["s1-feats"] = "_".join([f for f in stack[-2]["feats"].values()])
        else:
            features["s1-feats"] = False
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        # фічі для 1-го слова в черзі
        if queue_top["feats"] != None:
            features["q0-feats"] = "_".join([f for f in queue_top["feats"].values()])
        else:
            features["q0-feats"] = False
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
        # фічі для 2-го слова в черзі
        if queue_next["feats"] != None:
            features["q1-feats"] = "_".join([f for f in queue_next["feats"].values()])
        else:
            features["q1-feats"] = False
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

In [7]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []
    
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                       queue[0] if len(queue) > 0 else None,
                       relations)
        features.append(extract_features(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [8]:
train_features, train_labels = [], []
for tree in train_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

190298 190298


In [9]:
test_features, test_labels = [], []
for tree in test_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

25820 25820


In [10]:
from sklearn.feature_extraction import DictVectorizer 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

In [11]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)
print("\nTotal number of features:", len(vec.get_feature_names()))


Total number of features: 115303


In [12]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)
print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

190298 25820


In [13]:
lrc = LogisticRegression(random_state=42, solver="saga", multi_class="multinomial", max_iter=1000, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 644 epochs took 180 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='saga', tol=0.0001, verbose=1,
                   warm_start=False)

In [14]:
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

        left       0.87      0.89      0.88      6371
      reduce       0.86      0.81      0.83      6875
       right       0.78      0.81      0.79      5996
       shift       0.86      0.88      0.87      6578

    accuracy                           0.85     25820
   macro avg       0.85      0.85      0.85     25820
weighted avg       0.85      0.85      0.85     25820



In [15]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [16]:
logistic = LogisticRegression()
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=1)

In [17]:
# Fit grid search
best_model = clf.fit(train_features_vectorized, train_labels)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Us

/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    http

/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    http

/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/escapistdreamer/anaconda3/envs/spacy_nlp/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    http

In [18]:
# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l2
Best C: 1.0


In [19]:
pred_labels = best_model.predict(test_features_vectorized)
print(classification_report(test_labels, pred_labels))

              precision    recall  f1-score   support

        left       0.87      0.89      0.88      6371
      reduce       0.86      0.81      0.83      6875
       right       0.78      0.81      0.80      5996
       shift       0.87      0.88      0.87      6578

    accuracy                           0.85     25820
   macro avg       0.85      0.85      0.85     25820
weighted avg       0.85      0.85      0.85     25820



In [20]:
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [22]:
mod = GradientBoostingClassifier()
params = {'max_depth'    : [4, 6],
          'n_estimators' : [100, 500]  
         }

grid = GridSearchCV(estimator=mod, param_grid = params, cv = 2, n_jobs=-1)
grid.fit(train_features_vectorized, train_labels)

# Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")
print("\n The best estimator across ALL searched params:\n",
      grid.best_estimator_)
print("\n The best score across ALL searched params:\n",
      grid.best_score_)
print("\n The best parameters across ALL searched params:\n",
      grid.best_params_)
print("\n ========================================================")


 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

 The best score across ALL searched params:
 0.8712965979673986

 The best parameters across ALL searched params:
 {'max_depth': 6, 'n_estimators': 500}



In [23]:
predicted = grid.predict(test_features_vectorized)

In [24]:
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

        left       0.89      0.94      0.91      6371
      reduce       0.88      0.83      0.85      6875
       right       0.84      0.83      0.83      5996
       shift       0.90      0.92      0.91      6578

    accuracy                           0.88     25820
   macro avg       0.88      0.88      0.88     25820
weighted avg       0.88      0.88      0.88     25820

